In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from werkzeug.wrappers import Request, Response
from flask import Flask, request, jsonify, render_template
from tensorflow.keras.models import load_model
import pickle

C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

In [ ]:
app = Flask(__name__)

model = load_model('model.pkl')
scaler = pickle.load(open('scaler.pkl', 'rb'))
cols=['Bedroom','Bathroom','SquareFeet','Carpark','Type','Title','Oth_Info','State']

@app.route('/')
def home():
    return render_template('index.html')


@app.route('/predict',methods=['POST'])
def predict():
        
    int_features = [x for x in request.form.values()]
    
    final_features = np.array(int_features)
    
    data_unseen=pd.DataFrame([final_features],columns=cols)
    
    def type_label (row):
        if row['Type'] == 'Apartments':
            return 0
        if row['Type'] == 'Houses' :
            return 1

    def title_label (row):
        if row['Title'] == 'Freehold':
            return 0
        if row['Title'] == 'Leasehold' :
            return 1

    def oth1_label (row):
        if row['Oth_Info'] == 'Non Bumi Lot':
            return 0
        if row['Oth_Info'] == 'Bumi Lot':
            return 0
        if row['Oth_Info'] == 'Malay Reserved' :
            return 1

    def oth2_label (row):
        if row['Oth_Info'] == 'Malay Reserved':
            return 0
        if row['Oth_Info'] == 'Bumi Lot':
            return 0
        if row['Oth_Info'] == 'Non Bumi Lot' :
            return 1

    def state_label (row):
        if row['State'] == 'Kuala Lumpur':
            return 0
        if row['State'] == 'Selangor' :
            return 1

    data_unseen['Type_ Houses']=data_unseen.apply (lambda row: type_label(row), axis=1)
    data_unseen['Title_Leasehold']=data_unseen.apply (lambda row: title_label(row), axis=1)
    data_unseen['Oth_Info_Malay Reserved']=data_unseen.apply (lambda row: oth1_label(row), axis=1)
    data_unseen['Oth_Info_Non Bumi Lot']=data_unseen.apply (lambda row: oth2_label(row), axis=1)
    data_unseen['State_Selongor']=data_unseen.apply (lambda row: state_label(row), axis=1)

    data_unseen=data_unseen.drop(['Type','Title','Oth_Info','State'],axis=1)
    
    data_unseen=data_unseen.values
    
    #sample=np.array([4,3,2380,0,1,0,0,1,0])
    
    #data_unseen=scaler.fit_transform(data_unseen.reshape(-1,9))
    data_unseen=scaler.transform(data_unseen.reshape(-1,9))
    
    prediction = model.predict(data_unseen)

    output = int(prediction[0])

    return render_template('index.html', prediction_text='The Predicted House Price is: RM {}'.format(output))


if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple("localhost",9000,app)
   

Instructions for updating:
Colocations handled automatically by placer.


 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2020 20:19:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2020 20:19:46] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2020 20:19:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2020 20:19:59] "POST /predict HTTP/1.1" 200 -
